In [32]:
import os
import json
import requests
from datetime import datetime, timedelta
from amadeus import Client, ResponseError
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

# Load environment variables
load_dotenv()

# Initialize Amadeus Client
amadeus = Client(
    client_id=os.getenv('AMADEUS_KEY'),
    client_secret=os.getenv('AMADEUS_SECRET')
)
print(os.getenv('AMADEUS_SECRET'))

# Initialize LLM
llm = ChatOpenAI(temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))
def fetch_flight_inspirations(origin, departure_date=None, max_price=None):
    """
    Fetches possible destinations with indicative prices using Amadeus Flight Inspiration Search API.
    """
    try:
        # Prepare parameters based on the API's requirements
        params = {"origin": origin}
        if departure_date:
            params["departureDate"] = departure_date  # Use camelCase
        if max_price:
            params["maxPrice"] = max_price  # Correct camelCase for maxPrice

        # Call the Amadeus API
        response = amadeus.shopping.flight_destinations.get(**params)

        # Check and parse the response
        if response.status_code == 200:
            destinations = response.data
            return [
                {
                    "destination": destination["destination"],
                    "price": destination["price"]["total"],
                    "departure_date": destination.get("departureDate", "N/A")
                }
                for destination in destinations
            ]
        else:
            print(f"Unexpected response: {response.status_code} - {response.body}")
            return {"error": f"Unexpected API response: {response.status_code}"}

    except ResponseError as error:
        print(f"Error fetching flight inspirations: {error}")
        return {"error": f"Amadeus API error: {error}"}

def extract_flight_inspiration_parameters(user_query):
    """
    Uses the LLM to extract parameters for the Flight Inspiration Search API from the user's query.
    """
    today_date = datetime.today().strftime("%Y-%m-%d")

    prompt = PromptTemplate(
        input_variables=["query", "today_date"],
        template="""
Extract flight search parameters for inspiration from the following user query.
Return the response as a strict JSON object with the keys: origin, departure_date, max_price.

- The 'origin' key must contain a valid IATA code (e.g., 'JFK').
- The 'departure_date' key must be in 'YYYY-MM-DD' format only. Resolve relative terms like 'next week' into an exact date.
- The 'max_price' key is optional. If not mentioned, return null.

DO NOT include any additional text or comments in the response.

User Query: {query}
Today's Date: {today_date}
"""
    )

    formatted_prompt = prompt.format(query=user_query, today_date=today_date)
    response = llm.predict(formatted_prompt)
    response = eval(response.replace('null', 'None'))  # Convert JSON-like string to Python dict
    
    return response

def llm_agent_query_inspirations(user_query):
    """
    Integrates the LLM for parameter extraction and fetches flight inspirations using Amadeus API.
    """
    # Step 1: Extract parameters from user query
    params = extract_flight_inspiration_parameters(user_query)
    
    # Validate extracted parameters
    required_keys = ['origin', 'departure_date']
    if not all(params.get(key) for key in required_keys):
        return {"error": "Missing required parameters in query. Please provide origin and departure date."}
    print(params)

    # Step 2: Fetch flight inspirations
    inspirations = fetch_flight_inspirations(
        origin=params['origin'],
        departure_date=params['departure_date'],
        max_price=params.get('max_price')
    )

    return inspirations

# Define the Flight Inspiration Tool
flight_inspiration_tool = Tool(
    name="Flight Inspiration Search",
    func=llm_agent_query_inspirations,
    description=(
        "Fetch possible travel destinations with indicative prices based on user queries. "
        "Provide origin, departure date, and optionally, a maximum price. "
        "Returns a list of destinations with prices and dates."
    )
)



FndkeQqoOOswu7ek


In [33]:
# Example Usage
user_query = "Find me travel destinations from MAD in the next 2 months under 200 dollars"
result = flight_inspiration_tool.run(user_query)

# Output the results
if isinstance(result, dict) and "error" in result:
    print(result["error"])
else:
    for destination in result:
        print(f"Destination: {destination['destination']}, "
              f"Price: ${destination['price']}, "
              f"Departure Date: {destination['departure_date']}")

{'origin': 'MAD', 'departure_date': '2025-03-13', 'max_price': 200}
Destination: OPO, Price: $56.30, Departure Date: 2025-03-13
Destination: LIS, Price: $57.42, Departure Date: 2025-03-13
Destination: PMI, Price: $60.49, Departure Date: 2025-03-13
Destination: LPA, Price: $69.89, Departure Date: 2025-03-13
Destination: MXP, Price: $77.35, Departure Date: 2025-03-13
Destination: FCO, Price: $77.79, Departure Date: 2025-03-13
Destination: ORY, Price: $90.19, Departure Date: 2025-03-13
Destination: LGW, Price: $97.97, Departure Date: 2025-03-13
Destination: AMS, Price: $157.43, Departure Date: 2025-03-13
Destination: RAK, Price: $181.60, Departure Date: 2025-03-13


In [34]:

def fetch_historical_temperature(lat: float, lon: float, date: datetime) -> float:
    """
    Fetch historical temperature for a specific date and location using the OpenWeather /3.0/onecall/timemachine endpoint.
    """
    base_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine"
    params = {
        "lat": lat,
        "lon": lon,
        "dt": int(date.timestamp()),  # Convert datetime to UNIX timestamp
        "appid": API_KEY,
        "units": "metric"  # Use metric units (Celsius)
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data["data"][0]["temp"]  # Adjust the JSON path to fetch temperature
    except requests.exceptions.RequestException as e:
        print(f"Error fetching historical data: {e}")
        return None
    except KeyError:
        print("Temperature data not found in the response.")
        return None

def get_city_coordinates(city: str) -> tuple:
    """
    Fetch latitude and longitude for a city using the OpenWeather API.
    """
    geo_url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {
        "q": city,
        "limit": 1,
        "appid": API_KEY
    }

    response = requests.get(geo_url, params=params)
    if response.status_code == 200 and len(response.json()) > 0:
        data = response.json()[0]
        return data["lat"], data["lon"]
    else:
        print(f"Error fetching coordinates for {city}: {response.status_code} - {response.text}")
        return None, None

def weather_data_fetcher(inputs):
    """
    Fetches average temperature over the last 2 years for a location identified by IATA code
    from a specific target date. Expects a dictionary input with keys 'iata_code' and 'target_date'.
    """
    print(f"Type of inputs at start: {type(inputs)}")
    print(f"Value of inputs at start: {inputs}")
    ...
    if isinstance(inputs,dict):
        pass
    else:
        try:
            inputs = json.loads(inputs)
        except json.JSONDecodeError:
            return "Error: Input must be a valid JSON string."
    
    # Extract required fields
    iata_code = inputs.get("iata_code")
    target_date = inputs.get("target_date")
    # Validate target_date format
    try:
        target_date = datetime.strptime(target_date, "%Y-%m-%d")
    except ValueError:
        return f"Error: Invalid date format for target_date: {target_date}. Please use YYYY-MM-DD format."

    # Simulate fetching coordinates from IATA code
    lat, lon = get_city_coordinates(iata_code)
    if not lat or not lon:
        return f"Error: Unable to fetch coordinates for IATA code {iata_code}."

    # Fetch historical temperatures
    temperatures = []
    for year_offset in range(1, 3):  # Last 2 years relative to target_date
        historical_date = target_date - timedelta(days=365 * year_offset)
        temp = fetch_historical_temperature(lat, lon, historical_date)
        if temp is not None:
            temperatures.append(temp)

    if temperatures:
        avg_temp = sum(temperatures) / len(temperatures)
        return f"The average temperature at {iata_code} over the last 2 years from {target_date.strftime('%Y-%m-%d')} is {avg_temp:.2f}°C."
    else:
        return f"Error: Unable to fetch historical weather data for {iata_code}."




In [35]:

weather_tool = Tool(
    name="Weather Fetcher",
    func=weather_data_fetcher,
    description=(
        "Fetch historical weather data for a location identified by IATA code "
        "(e.g., MAD for Madrid) and a specific target date (YYYY-MM-DD). "
        "Input must be a JSON string with keys 'iata_code' and 'target_date'."
    )
)
def get_current_datetime():
    """
    Returns the current date and time in a standardized format (YYYY-MM-DD).
    """
    return datetime.now().strftime("%Y-%m-%d")

datetime_tool = Tool(
    name="Current DateTime Fetcher",
    func=lambda _: get_current_datetime(),
    description="Fetches the current date in 'YYYY-MM-DD' format. Useful for resolving relative terms like 'next week' or 'tomorrow'."
)

In [36]:
# Register tools
tools = [flight_inspiration_tool, weather_tool,datetime_tool]

# Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [37]:
query = """
Suggest a travel destination from Madrid with warm weather on 15-02-2025
"""

In [38]:

response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
We need to find a travel destination with warm weather on a specific date, so we should start by fetching weather data for Madrid on 15-02-2025 to determine what warm weather is in that context.

Action: Weather Fetcher
Action Input: {"iata_code": "MAD", "target_date": "2025-02-15"}Type of inputs at start: <class 'str'>
Value of inputs at start: {"iata_code": "MAD", "target_date": "2025-02-15"}

Observation: The average temperature at MAD over the last 2 years from 2025-02-15 is 1.48°C.
Thought:The weather data for Madrid on that date is not warm, so we should look for a destination with warmer weather.

Action: Flight Inspiration Search
Action Input: user_query("warm weather", "MAD", "2025-02-15"){'origin': 'MAD', 'departure_date': '2025-02-15', 'max_price': None}

Observation: [{'destination': 'LIS', 'price': '57.42', 'departure_date': '2025-02-15'}, {'destination': 'OPO', 'price': '58.30', 'departure_date': '2025-02-15'}, {'destination': 'PMI'